In [14]:
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from typing import TypedDict,Annotated
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os 

In [15]:
class base_state(TypedDict):
    query:str
    gr_rem:str
    final_prompt:str

File exists: False
Absolute path: /content/Backend/Agents/.env


In [ ]:
load_dotenv()
def get_llm():
    llm= ChatGroq(api_key=os.getenv("GROQ_API_KEY"),temperature=0.5,model="openai/gpt-oss-120b")
    return llm

In [ ]:
async def grammer(state:base_state):
    "remove grammer mistake from query"
    prompt = PromptTemplate(
        input_variables=["query"],
        template=""" 
        clean the following prompt
      remove the grammer_mistake and make it clear

query:
{query}

"""
    )
    llm =  get_llm()
    chain=  prompt | llm | StrOutputParser()

    result =   await chain.ainvoke({
        "query":state["query"]
    })
    return {
        "gr_rem":result
    }


In [ ]:
async def prompt_enh(state:base_state):
    "enhance the  prompt "
    prompt = PromptTemplate(
        input_variables=["gr_rem"],
        template=""" 
Task:
You are a prompt engineering expert. Your goal is to transform the Input Prompt into three distinct, high-quality prompt versions, each clearly labeled and written in professional, precise language.

Input:
{gr_rem}

Requirements:

Rewrite the input prompt; do not answer or execute it.

Each version must be self-contained, unambiguous, and optimized for use with an LLM.

Use clear structure, explicit instructions, and strong constraints.

Maintain the original intent of {gr_rem} while improving clarity and effectiveness.

Deliverables (provide all three):

Detailed Prompt

Multiple paragraphs

Clearly state the objective, context, constraints, assumptions, and expected output

Include formatting or style requirements if relevant

Concise Prompt

A shorter, highly focused version

Retains the core objective and key constraints

Optimized for quick use while remaining precise

Instructional Prompt

Written as step-by-step instructions to the model

Emphasizes process, reasoning expectations, and output structure

Output Format (use exactly this structure):

Detailed Prompt:
<your detailed prompt here>

Concise Prompt:
<your concise prompt here>

Instructional Prompt:
<your instructional prompt here> 

"""
    )
    llm =  get_llm()
    chain = prompt | llm | StrOutputParser()

    result = await chain.ainvoke({
        "gr_rem":state["gr_rem"]
    })
    return {
        "final_prompt":result
    }

In [ ]:
graph= StateGraph(base_state)

In [ ]:
graph.add_node("mistake_remover",grammer)
graph.add_node("prompt_enhancer",prompt_enh)

In [ ]:
graph.add_edge(START,"mistake_remover")
graph.add_edge("mistake_remover","prompt_enhancer")
graph.add_edge("prompt_enhancer",END)

agent = graph.compile()

In [ ]:
intial_state={
    "query":"what is jwt"
}

**Cleaned Prompt**

Please clean the following prompt by removing any grammatical mistakes and making it clear.

**Query:**  
What is JWT?**Detailed Prompt:**  
You are an expert editor tasked with refining a user‑submitted prompt.  

**Objective:** Rewrite the supplied prompt so that it is free of grammatical errors, unambiguous, and maximally clear, while preserving its original intent.  

**Context:** The original prompt asks the model to “clean the following prompt by removing any grammatical mistakes and making it clear.” The embedded query to be cleaned is:  

```
What is JWT?
```  

**Constraints:**  
1. Do not answer the query “What is JWT?” – only edit the surrounding instruction.  
2. Preserve the structure of the original request (i.e., a brief instruction followed by the query).  
3. Maintain a professional tone and use standard English grammar.  
4. Keep the cleaned prompt concise but fully understandable.  

**Assumptions:** The user expects the cleaned prompt to be ready